In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import os,sys 
import glob

import plotly.graph_objects as go
import plotly.offline as po
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from matplotlib import dates, colors

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.lift()
root.withdraw()
scirun_dir = filedialog.askdirectory(title='SCIRun Directory') #Select SCIRun directory

global FiberActivationFile

files = glob.glob(scirun_dir + '/simulations/*.csv')

name_list = []
for file in files:
    name_list.append(file.split('\\')[-1].split('.')[0].replace('_',' ').title().replace('Activation ', ''))
    
@interact
def file_select(Filename=name_list):
    file_loc = [i for i, file in enumerate(name_list) if Filename in file]
    global FiberActivationFile
    FiberActivationFile = files[file_loc[0]]


interactive(children=(Dropdown(description='Filename', options=('Left Pallidum Directional', 'Left Pallidum Ri…

In [42]:
#FiberActivationFile = r"D:\Dropbox (UFL)\Projects\Tourette DBS UH3\TSDBS_Data\S2\Image_processing\SCIRun\simulations\activation_Left_pallidum_directional.csv"
FiberActivations = pd.read_csv(FiberActivationFile)
availableFibers = sorted(np.unique(FiberActivations["Fiber_Name"]))

@interact
def show_activation_of(fiberName=availableFibers, amplitude=widgets.IntSlider(min=1, max=5, step=1, value=1)):
    activationValues = FiberActivations.loc[FiberActivations["Fiber_Name"] == fiberName].loc[FiberActivations["Amplitude"] == amplitude]
    fig = make_subplots(rows=1, cols=2, shared_xaxes=True, shared_yaxes=False,
                        subplot_titles=[(fiberName.split('_')[1]+ ' ' + fiberName.split('_')[2] if 'Sum' in fiberName else str(('Left ' if 'L' in fiberName.split('_')[1] else 'Right ') + fiberName.split('_')[0])) + f" activation at {amplitude}mA", (fiberName.split('_')[1]+ ' ' + fiberName.split('_')[2] if 'Sum' in fiberName else str(('Left ' if 'L' in fiberName.split('_')[1] else 'Right ') + fiberName.split('_')[0])) + f" Activation Heatmap"])

    if len(activationValues["Contact"]) == 8:
        ContactNames = ["E00", "E01A", "E01B", "E01C", "E02A", "E02B", "E02C", "E03"]
    elif len(activationValues["Contact"]) == 4:
        ContactNames = ["E00", "E01", "E02", "E03"]

    fig.add_trace(
        go.Bar(x=ContactNames, y=activationValues["Percent_Activation"],
                   hovertemplate="%{x}: %{y:.1f}% <extra></extra>",
                   showlegend=False
                  ),
        row=1, col=1
    )

    if len(activationValues["Contact"]) == 8:
        activationHeatmap = np.zeros((4,5))
        activationHeatmap[0,:] = activationValues.loc[FiberActivations['Contact'] == 0]["Percent_Activation"]
        activationHeatmap[1,0] = activationValues.loc[FiberActivations['Contact'] == 3]["Percent_Activation"]
        activationHeatmap[1,4] = activationValues.loc[FiberActivations['Contact'] == 1]["Percent_Activation"]
        activationHeatmap[1,1:4] = activationValues.loc[np.bitwise_and(FiberActivations['Contact'] > 0, FiberActivations['Contact'] < 4)]["Percent_Activation"]
        activationHeatmap[2,0] = activationValues.loc[FiberActivations['Contact'] == 6]["Percent_Activation"]
        activationHeatmap[2,4] = activationValues.loc[FiberActivations['Contact'] == 4]["Percent_Activation"]
        activationHeatmap[2,1:4] = activationValues.loc[np.bitwise_and(FiberActivations['Contact'] > 3, FiberActivations['Contact'] < 7)]["Percent_Activation"]
        activationHeatmap[3,:] = activationValues.loc[FiberActivations['Contact'] == 7]["Percent_Activation"]
        fig.add_trace(
            go.Heatmap(x=[-1,0,1,2,3], y=[0,1,2,3], z=activationHeatmap,
                       type="heatmap", zsmooth="best", colorscale="jet", hovertemplate="%{z:.2f}% <extra></extra>",
                       zmin=0, zmax=100,
                       showlegend=False
                      ),
            row=1, col=2
        )
        
        fig.update_xaxes(title_font_size=15, title_text="", tickfont_size=12, range=(-0.5,2.5),
                         color="#000000", ticks="outside", showline=True, linecolor="#DDDDDD", 
                         tickvals=[0,1,2], ticktext=["Segment A", "Segment B", "Segment C"],
                         tickangle=-30,
                         showgrid=True, gridcolor="#DDDDDD", col=2, row=1)
        
        fig.update_yaxes(type='linear',
                     title_font_size=15, title_text="", tickfont_size=12,
                     tickmode="array", tickvals=[0,1,2,3], ticktext=["E00","E01","E02","E03"],
                     color="#000000", ticks="outside", showline=True, linecolor="#000000", zeroline=False, zerolinecolor="#000000",
                     showgrid=True, gridcolor="#DDDDDD", col=2, row=1)
    
    elif len(activationValues["Contact"]) == 4:
        activationHeatmap = np.zeros((4,1))
        activationHeatmap[0,:] = activationValues.loc[FiberActivations['Contact'] == 0]["Percent_Activation"]
        activationHeatmap[1,:] = activationValues.loc[FiberActivations['Contact'] == 1]["Percent_Activation"]
        activationHeatmap[2,:] = activationValues.loc[FiberActivations['Contact'] == 2]["Percent_Activation"]
        activationHeatmap[3,:] = activationValues.loc[FiberActivations['Contact'] == 3]["Percent_Activation"]
        fig.add_trace(
            go.Heatmap(x=[0], y=[0,1,2,3], z=activationHeatmap,
                       type="heatmap", zsmooth=None, colorscale="jet", hovertemplate="%{z:.2f}% <extra></extra>",
                       zmin=0, zmax=100,
                       showlegend=False
                      ),
            row=1, col=2
        )
        
        fig.update_xaxes(title_font_size=15, title_text="", tickfont_size=12, range=(-0.5,0.5),
                         color="#000000", ticks="outside", showline=True, linecolor="#DDDDDD", 
                         tickvals=[0,1,2], ticktext=["Ring"],
                         tickangle=-30,
                         showgrid=True, gridcolor="#DDDDDD", col=2, row=1)
        
        fig.update_yaxes(type='linear',
                     title_font_size=15, title_text="", tickfont_size=12,
                     tickmode="array", tickvals=[0,1,2,3], ticktext=["E00","E01","E02","E03"],
                     color="#000000", ticks="outside", showline=True, linecolor="#000000", zeroline=False, zerolinecolor="#000000",
                     showgrid=True, gridcolor="#DDDDDD", col=2, row=1)


    fig.update_yaxes(type='linear', range=(0,100),
                     title_font_size=15, title_text="Percent Activation (%)", tickfont_size=12,
                     tickmode="array", tickvals=np.arange(0,120,20), ticktext=[f"{i}" for i in np.arange(0,120,20)],
                     color="#000000", ticks="outside", showline=True, linecolor="#000000", zeroline=False, zerolinecolor="#000000",
                     showgrid=True, gridcolor="#DDDDDD", col=1, row=1)

    fig.update_xaxes(title_font_size=15, title_text="Contact ID", tickfont_size=12,
                     color="#000000", ticks="outside", showline=True, linecolor="#DDDDDD", tickvals=ContactNames,
                     tickangle=-30,
                     showgrid=True, gridcolor="#DDDDDD", col=1, row=1)

    fig.layout["paper_bgcolor"]="#FFFFFF"
    fig.layout["plot_bgcolor"]="#FFFFFF"
    fig.layout["height"] = 400

    fig.show(renderer="notebook")

interactive(children=(Dropdown(description='fiberName', options=('Sum_Right_ACC_hdp.100', 'Sum_Right_M1_hdp_fa…

In [43]:
availableContacts = sorted(np.unique(FiberActivations["Contact"]))
if len(availableContacts) == 8:
    ContactNames = ["E00", "E01A", "E01B", "E01C", "E02A", "E02B", "E02C", "E03"]
elif len(availableContacts) == 4:
    ContactNames = ["E00", "E01", "E02", "E03"]
    
@interact
def show_activation_of(contactName=ContactNames, amplitude=widgets.IntSlider(min=1, max=5, step=1, value=1)):
    contactIndex = np.where([contactName == name for name in ContactNames])[0][0]
    activationValues = FiberActivations.loc[FiberActivations["Contact"] == contactIndex].loc[FiberActivations["Amplitude"] == amplitude]
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True, shared_yaxes=False,
                        subplot_titles=[f"{contactName} Fiber Activation at {amplitude}mA"])
    
    fig.add_trace(
        go.Bar(x=activationValues["Fiber_Name"], y=activationValues["Percent_Activation"],
                   hovertemplate="%{x}: %{y:.1f}% <extra></extra>",
                   showlegend=False
                  ),
        row=1, col=1
    )

    fig.update_yaxes(type='linear', range=(0,100),
                     title_font_size=15, title_text="Percent Activation (%)", tickfont_size=12,
                     tickmode="array", tickvals=np.arange(0,120,20), ticktext=[f"{i}" for i in np.arange(0,120,20)],
                     color="#000000", ticks="outside", showline=True, linecolor="#000000", zeroline=False, zerolinecolor="#000000",
                     showgrid=True, gridcolor="#DDDDDD", col=1, row=1)

    fig.update_xaxes(title_font_size=15, title_text="Fiber Names", tickfont_size=12,
                     color="#000000", ticks="outside", showline=True, linecolor="#DDDDDD", tickvals=activationValues["Fiber_Name"],
                     tickangle=-30,
                     showgrid=True, gridcolor="#DDDDDD", col=1, row=1)

    fig.layout["paper_bgcolor"]="#FFFFFF"
    fig.layout["plot_bgcolor"]="#FFFFFF"
    fig.layout["height"] = 600

    fig.show(renderer="notebook")

interactive(children=(Dropdown(description='contactName', options=('E00', 'E01A', 'E01B', 'E01C', 'E02A', 'E02…